In [1]:
import pandas as pd
from scipy import stats
import numpy as np

### まずデータを読み込む

In [2]:
df =pd.read_csv("data.csv")
df.head()

,filename,ID,age,sex,LR,AMD,RVO,Gla,MH,DR,RD,RP,AO,DM
0,000000_00.jpg,3090,78,M,L,0,0,0,0,0,0,0,0,0
1,000000_01.jpg,3090,78,M,R,0,0,0,0,0,0,0,0,0
2,000001_00.jpg,2702,76,F,L,0,0,0,0,0,0,0,0,0
3,000001_01.jpg,2702,76,F,R,0,0,0,0,0,0,0,0,0
4,000001_02.jpg,1136,69,M,L,0,0,1,0,0,0,0,0,0


# 2クラスの比率比較
Fisherの正確検定を行う<br>
np < 5だとX<sup>2</sup>検定だと行えないみたいな仮定が必要ない

### AMD群とRVO群の女性比率を比較する

In [4]:
# データの読み込み
sex0 = df[df["AMD"] == 1]["sex"]
sex1 = df[df["RVO"] == 1]["sex"]

### テンプレート化
比較したいデータを二つおよび、なんという内容が出てくる比率を比較するかを検定するかを指定する。<br>
k0, n0, 比率が2群とも、およびFisherの正確検定を行ったp値が出力される

In [3]:
def compare_ratio_2group(val0, val1, target, digit = 3, digit_p = 3):
    n0 = len(val0)
    k0 = np.sum(val0 == target)
    n1 = len(val1)
    k1 = np.sum(val1 == target)
    res = stats.fisher_exact([[k0, n0-k0], [k1, n1-k1]])
    return [k0, n0, round(k0/n0, digit)], [k1, n1, round(k1/n1, digit)], round(res[1], digit_p)

In [5]:
compare_ratio_2group(sex0, sex1, "F")

([110, 413, 0.266], [367, 778, 0.472], 0.0)

# 3クラスの比率比較
Fisherの正確検定を行う<br>
その際に、p値を補正する。
Bonferroni, Holm, Hochberg法というのがあるが、Hochberg法を採用

### AMD群とRVO, DR群の性別を比較する

### テンプレート化
よく使うであろう多クラス間の比率の値、p値を算出

In [62]:
# よくあるk, n, k/n(比率)を求める
# tagと値の配列のペアとなっているdictionaryが投入される
# どの組み合わせで有意差があったか、タグと(k, n, k/n)のセット、Fisher exact test with
# Hochbergの結果DataFrameが出力される

def compare_ratio_multi(dics, target, digit = 3):
    tags = list(dics.keys())
    vals = {}
    for tag in tags:
        n = len(dics[tag])
        k = np.sum(dics[tag] == target)
        vals[tag] = (n, k, round(k/n, digit))    
    
    datas = []
    for i1, tag1 in enumerate(tags):
        for i2, tag2 in enumerate(tags):
            if i1 < i2:
                    n1 = len(dics[tag1])
                    k1 = np.sum(dics[tag1] == target)
                    n2 = len(dics[tag2])
                    k2 = np.sum(dics[tag2] == target)
                    res = stats.fisher_exact([[k1, n1-k1], [k2, n2-k2]])[1]
                    datas.append([tag1, tag2, res])
    df_result = pd.DataFrame(data=datas, columns=["Tag1", "Tag2", "p-value"])
    df_result = df_result.sort_values(by="p-value", ascending=True).reset_index(drop=True)
    ps = df_result["p-value"]
    n = len(ps)
    newps = [p*(n-i) for i, p in enumerate(ps)]
    df_result["p-value adjust"] = newps
    res = np.any(np.array(newps) < 0.05)
    return res, vals, df_result

In [63]:
df =pd.read_csv("data.csv")
target = "F"
digit = 1
dics = {}
tags = ["AMD", "RVO", "DR"]
for tag in tags:
    dics[tag] = df[df[tag] == 1]["sex"]
compare_ratio_multi(dics, target)

(True,
 {'AMD': (413, 110, 0.266),
  'DR': (3323, 1287, 0.387),
  'RVO': (778, 367, 0.472)},
   Tag1 Tag2       p-value  p-value adjust
 0  AMD  RVO  4.151636e-12    1.245491e-11
 1   DR  AMD  1.128645e-06    2.257290e-06
 2   DR  RVO  1.973059e-05    1.973059e-05)